#### The other day I was listening to [Linear Digressions podcast](http://lineardigressions.com/) and they were talking about a tool to visualize numerical features called Chernoff faces ([episode](http://lineardigressions.com/episodes/2016/2/29/chernoff-faces-and-minard-maps)). I wanted to give it a try in this dataset just for fun.

### What are Chernoff faces?
#### Chernoff faces, invented by Herman Chernoff in 1973, display multivariate data in the shape of a human face. The individual parts, such as eyes, ears, mouth and nose represent values of the variables by their shape, size, placement and orientation. The idea behind using faces is that humans easily recognize faces and notice small changes without difficulty. Chernoff faces handle each variable differently. Because the features of the faces vary in perceived importance, the way in which variables are mapped to the features should be carefully chosen (e.g. eye size and eyebrow-slant have been found to carry significant weight) ([paper]( http://www.research.ibm.com/people/c/cjmorris/publications/Chernoff_990402.pdf)). Following is an example for the famous iris dataset ([source](https://archive.ics.uci.edu/ml/datasets/iris)) where features (sepal length, sepal width, petal length, petal width) are presented as Chernoff faces and the colour marks each of the species (i.e. the target: setosa, versicolor, virginica) ([source](https://github.com/antononcube/MathematicaForPrediction/blob/master/MarkdownDocuments/Making-Chernoff-faces-for-data-visualization.md)):

![Chernoff faces](https://camo.githubusercontent.com/ced6cd0923a923aade542c33f9ccc86614049e30ab7c0c1085ee7823752ce858/687474703a2f2f692e696d6775722e636f6d2f7550425a4a75666c2e676966)

#### It can be seen that different species have different characteristic facial features that let their indentification visually very easily.
#### Let's take a look now at how the Chernoff faces look like for the numerical features of this competition and whether the resultant faces can help to discriminate the target classes in some way.

In [ ]:
import numpy as np 
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import os


Мы хотим нормализацию что бы значения от 0 до 100 оказывали сильное вли

In [ ]:
x = np.linspace(1, 500)
max=np.pi/2
plt.plot(x, np.arctan(x/30)/max)
plt.show()

#np.arctan(2/10)/max/2

In [ ]:
# Function to draw Chernoff faces (source: https://gist.github.com/dribnet/e26f52f423f0656c1bb8fc6f4e741cc2#file-mpl_cfaces-py)

def cface(ax, x1,x2,x3,x4,x5,x6,x7,x8,x9,x10,x11,x12,x13,x14,x15,x16,x17,x18):
    # x1 = height  of upper face
    # x2 = overlap of lower face
    # x3 = half of vertical size of face
    # x4 = width of upper face
    # x5 = width of lower face
    # x6 = length of nose
    # x7 = vertical position of mouth
    # x8 = curvature of mouth
    # x9 = width of mouth
    # x10 = vertical position of eyes
    # x11 = separation of eyes
    # x12 = slant of eyes
    # x13 = eccentricity of eyes
    # x14 = size of eyes
    # x15 = position of pupils
    # x16 = vertical position of eyebrows
    # x17 = slant of eyebrows
    # x18 = size of eyebrows
    
    # transform some values so that input between 0,1 yields variety of output
    
    x3 = 1.9*(x3-.5)
    x4 = (x4+.25)
    x5 = (x5+.2)
    x6 = .3*(x6+.01)
    x8 = 5*(x8+.001)
    x11 /= 5
    x12 = 2*(x12-.5)
    x13 += .05
    x14 += .1
    x15 = .5 *(x15-.5)
    x16 = .25*x16
    x17 = .5*(x17-.5)
    x18 = .9*(x18+.1)

    # top of face, in box with l=-x4, r=x4, t=x1, b=x3
    e = matplotlib.patches.Ellipse( (0,(x1+x3)/2), 2*x4, (x1-x3), fc='white', edgecolor='black', linewidth=2)
    #e.set_clip_box(ax.bbox)
    #e.set_facecolor([0,0,0])
    ax.add_artist(e)

    # bottom of face, in box with l=-x5, r=x5, b=-x1, t=x2+x3
    e = matplotlib.patches.Ellipse( (0,(-x1+x2+x3)/2), 2*x5, (x1+x2+x3), fc='white', edgecolor='black', linewidth=2)
    ax.add_artist(e)

    # cover overlaps
    e = matplotlib.patches.Ellipse( (0,(x1+x3)/2), 2*x4, (x1-x3), fc='white', edgecolor='black', ec='none')
    ax.add_artist(e)
    e = matplotlib.patches.Ellipse( (0,(-x1+x2+x3)/2), 2*x5, (x1+x2+x3), fc='white', edgecolor='black', ec='none')
    ax.add_artist(e)
    
    # draw nose
    ax.plot([0,0], [-x6/2, x6/2], 'b', lw =5)
    
    # draw mouth
    p = matplotlib.patches.Arc( (0,-x7+.5/x8), 1/x8, 1/x8, theta1=270-180/np.pi*np.arctan(x8*x9), theta2=270+180/np.pi*np.arctan(x8*x9), lw=5)
    ax.add_artist(p)
    
    # draw eyes
    p = matplotlib.patches.Ellipse( (-x11-x14/2,x10), x14, x13*x14, angle=-180/np.pi*x12, facecolor='m', edgecolor='m')
    ax.add_artist(p)
    #ax.fill_between( -x11-x14/2,x10, "red")       
    p = matplotlib.patches.Ellipse( (x11+x14/2,x10), x14, x13*x14, angle=180/np.pi*x12, facecolor='m', edgecolor='m')
    ax.add_artist(p)

    # draw pupils
    p = matplotlib.patches.Ellipse( (-x11-x14/2-x15*x14/2, x10), .09, .09, facecolor='g')
    ax.add_artist(p)
    p = matplotlib.patches.Ellipse( (x11+x14/2-x15*x14/2, x10), .09, .09, facecolor='g')
    ax.add_artist(p)
    
    # draw eyebrows
    #ax.plot([-x11-x14/2-x14*x18/2,-x11-x14/2+x14*x18/2],[x10+x13*x14*(x16+x17)+0.1,x10+x13*x14*(x16-x17)+0.1],'r', lw=3)
    #ax.plot([x11+x14/2+x14*x18/2,x11+x14/2-x14*x18/2],[x10+x13*x14*(x16+x17)+0.1,x10+x13*x14*(x16-x17)+0.1],'r', lw=3)

# тут наши данные
!rm -rf img
!mkdir -p img

#cface(ax,  0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1 )
# [0,1,2,3,5,8,9,10,11,12,15] - most significant columns
sites=[ "Amazoncom", "Grocerywalmartcom", "Instacartcom", "Walmartcom", "Petcocom", "Meijercom", "Targetcom", "Homedepotcom", "Chewycom", "Petsmartcom", "Krogercom"]
values = [ 0, 0, 0, 0,0, 0, 0, 0, 0, 0, 0 ]

report = {}
i=0
for task in sites:
    report[task] = values[i]
    i=i+1
    
max_listing=500
    
norm_report={}

default_val_1=0.5

for default_val in np.linspace(0.62, 1, num=5):
#real data
    max=np.pi
    fig = plt.figure()
    ax = fig.add_subplot(1,1,1,aspect='equal')

    for site, num  in report.items():
      norm_report[site]=default_val+np.arctan(num/20)/max
        
   
    columns = [0,1,2,3,5,8,9,10,11,12,15] 
    iter=0
    face_values= [default_val_1] * 18
    for site, num in norm_report.items():
        face_values[columns[iter]]=num
        iter=iter+1



    cface(ax,*face_values)
    ax.axis([-1.2,1.2,-1.2,1.2])
    ax.set_facecolor('xkcd:turquoise')

    ax.set_xticks([])
    ax.set_yticks([])
    fig.subplots_adjust(hspace=0, wspace=0)
    #fig.text(0,0,default_val)
    plt.savefig('img/predicted' + str(default_val) + '.png', bbox_inches='tight')
#norm_report




In [ ]:
!zip  /kaggle/working/work.zip /kaggle/working/img/*

In [ ]:
import io


In [ ]:
buf = io.BytesIO()
fig.savefig(buf)
buf.seek(0)




In [ ]:
trfig, ax = plt.subplots(1,1, figsize=(20,15))

for i in range(1):
    for j in range(1):
        if j < 5:
            cface(ax[i,j], .9, *cf_zeros[np.random.randint(cf_zeros.shape[0]),:])
            ax[i,j].set_facecolor('xkcd:salmon')
        else:
            cface(ax[i,j], .9, *cf_ones[np.random.randint(cf_ones.shape[0]),:])
            ax[i,j].set_facecolor('xkcd:turquoise')
        ax[i,j].axis([-1.2, 1.2, -1.2, 1.2])
        ax[i,j].set_xticks([])
        ax[i,j].set_yticks([])

#### I leave it to you to find the characteristic facial features (if any) of each class!